In [1]:
import pema
%run init.py

ntauxfiles is not installed
Working on dali009.rcc.local with the following versions and installation paths:
python	v3.8.5	(default, Sep  4 2020, 07:30:14) [GCC 7.3.0]
strax	v0.15.1	/home/angevaare/software/strax/strax
straxen	v0.18.3	/home/angevaare/software/straxen/straxen
wfsim	v0.5.1	/home/angevaare/software/WFSim/wfsim
nestpy	v1.4.7	/home/angevaare/software/Miniconda3/envs/strax/lib/python3.8/site-packages/nestpy
pema	v0.1.0	/home/angevaare/software/pema/pema


In [2]:
%run setup_lowe.py

In [3]:
testing_config = {
#     'gcn_model': None,
#     'cnn_model': None
                 }

def get_context(skip=None):
    """
    Get a context and set all of the entries in the testing_config to the 
    context config IF they are not in 'skip' 
    """
    # Directories are set in the "%run setup_..." above
    st = pema.pema_context(base_dir=base_dir,
                           config_update=config_update,
                           raw_dir=raw_data_dir,
                           data_dir=data_dir)

    # Update the context config with all of the parameters that are not skipped
    for k, v in testing_config.items():
        if skip is not None and k in skip:
            print(f'skip {k}')
        else:
            st.set_config({k:v})

    # Don't allow lazy mode for any config
    st.set_context_config(
        {'allow_shm': True,
         'allow_lazy': False,
         'timeout': 600,
         'max_messages': 4,
        })
    return st

st = get_context()

In [4]:
r = run_list[0]
peaks = st.get_array(r, 'peak_basics')
truth = st.get_array(r, 'truth')

Loading peak_basics: |          | 0.00 % [00:00<?]

Loading truth: |          | 0.00 % [00:00<?]

In [5]:
len(peaks), len(truth)

(913420, 958026)

In [6]:
truth = pema.append_fields(truth, 'area', truth['n_photon'])

In [7]:
truth = pema.append_fields(
    truth,
    'id',
    np.arange(len(truth)),
    dtypes=np.int64)

In [8]:
peaks = pema.append_fields(
    peaks,
    'id',
    np.arange(len(peaks)),
    dtypes=np.int64)

In [9]:
truth['endtime'] = truth['t_last_photon'].copy()

In [10]:
# truth_vs_peak, peak_vs_truth = pema.match_peaks(truth, peaks)

In [11]:
len(truth), len(peaks)

(958026, 913420)

In [12]:
_max = 40_000
small_truth, small_peaks = truth[:_max].copy(), peaks[:_max].copy()

In [20]:
%%time
_max = 40_000
small_truth, small_peaks = truth[:_max].copy(), peaks[:_max].copy()
_ = pema.match_peaks(small_truth, small_peaks )

CPU times: user 1.98 s, sys: 2.67 ms, total: 1.98 s
Wall time: 2.07 s


In [21]:
%%time
_max = 80_000
small_truth, small_peaks = truth[:_max].copy(), peaks[:_max].copy()
_ = pema.match_peaks(small_truth, small_peaks )

CPU times: user 9.87 s, sys: 31.3 ms, total: 9.9 s
Wall time: 10.3 s


In [22]:
%%time
_max = 160_000
small_truth, small_peaks = truth[:_max].copy(), peaks[:_max].copy()
_ = pema.match_peaks(small_truth, small_peaks )

CPU times: user 41.1 s, sys: 241 ms, total: 41.3 s
Wall time: 43 s


In [23]:
%%time
_max = 320_000
small_truth, small_peaks = truth[:_max].copy(), peaks[:_max].copy()
_ = pema.match_peaks(small_truth, small_peaks )

CPU times: user 2min 40s, sys: 797 ms, total: 2min 41s
Wall time: 2min 49s


In [ ]:
%%time
_max = 640_000
small_truth, small_peaks = truth[:_max].copy(), peaks[:_max].copy()
_ = pema.match_peaks(small_truth, small_peaks )

In [ ]:
%%time
_ = pema.match_peaks(truth, peaks )

In [13]:
# from line_profiler import LineProfiler

# lp = LineProfiler()
# lp.add_function(pema.match_peaks) 
# lp.add_function(strax.processing.general._touching_windows)
# lp.add_function(pema.matching.get_deepwindows)
# lp.add_function(pema.matching._match_peaks)
# lp.add_function(pema.append_fields)

# lp_wrapper = lp(pema.match_peaks)

# lp_wrapper(small_truth, small_peaks)
# lp.print_stats()

In [14]:
"""
Utility to match peaks from results of different processor versions / processor and simulator
Joran Angevaare, october, november 2020
Joey Howlett, February 2020
Joran Angevaare, September 2019
Mike Clark, September 2019
Jelle Aalbers, Nikhef, September 2015
"""

import numpy as np
import pema
import strax
import numba
import logging

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(threadName)s - %(name)s - %(levelname)s - %(message)s')
log = logging.getLogger('Pema matching')

# export, __all__ = strax.exporter()

INT_NAN = -99999
OUTCOME_DTYPE = '<U32'


# @export
def match_peaks(allpeaks1, allpeaks2,
                matching_fuzz=0,
                unknown_types=(0,)):
    """
    Perform peak matching between two numpy record arrays with fields:
        Event, left, right, type, area
    If a peak is split into many fragments (e.g. two close peaks split
    into three peaks), the results are unreliable and depend on which
    peak set is peaks1 and which is peaks2.

    Returns (allpeaks1, allpeaks2), each with three extra fields:
    id, outcome, matched_to:
        id: unique number for each peak
        outcome: Can be one of:
            found:  Peak was matched 1-1 between peaks1 and peaks2 (type agrees,
             no other peaks in range).
                    Note that area, widths, etc. can still be quite different!
            missed: Peak is not present in the other list
            misid_as_XX: Peak is present in the other list, but has type XX
            merged: Peak is merged with another peak in the other list, the new
            'super-peak' has the same type
            merged_to_XX: As above, but 'super-peak' has type XX
            split: Peak is split in the other list, but more than one fragment
            has the same type as the parent.
            chopped: As split, but one or several fragments are unclassified,
            exactly one has the correct type.
            split_and_unclassified: As split, but all fragments are unclassified
            in the other list.
            split_and_misid: As split, but at least one fragment has a different
            peak type.
        matched_to: id of matching in *peak* in the other list if outcome is found
        or misid_as_XX, INT_NAN otherwise.
    """
    # Check required fields
    for i, d in enumerate((allpeaks1, allpeaks2)):
        assert hasattr(d, 'dtype'), 'Cannot work with non-numpy arrays'
        m = ''
        for k in ('area', 'type', 'id'):
            if k not in d.dtype.names:
                m += f'Argument {i} misses field {k} required for matching \n'
        if m != '':
            raise ValueError(m)
    log.debug('Appending fields')
    # Append id, outcome and matched_to fields
    allpeaks1 = pema.append_fields(
        allpeaks1,
        ('outcome', 'matched_to'),
        (np.array(['missed'] * len(allpeaks1), dtype=OUTCOME_DTYPE),
         INT_NAN * np.ones(len(allpeaks1), dtype=np.int64)),
        dtypes=(OUTCOME_DTYPE, np.int64),
    )
    allpeaks2 = pema.append_fields(
        allpeaks2,
        ('outcome', 'matched_to'),
        (np.array(['missed'] * len(allpeaks2), dtype=OUTCOME_DTYPE),
         INT_NAN * np.ones(len(allpeaks2), dtype=np.int64)),
        dtypes=(OUTCOME_DTYPE, np.int64),
    )

    log.debug('Getting windows')
    windows = strax.touching_windows(allpeaks1, allpeaks2, window=matching_fuzz)
    deep_windows = np.empty((0, 2), dtype=(np.int64, np.int64))
    # Each of the windows projects to a set of peaks in allpeaks2
    # belonging to allpeaks1. We also need to go the reverse way, which
    # I'm calling deep_windows below.
    if len(windows):
        return windows, allpeaks2, allpeaks1, matching_fuzz
        # The order matters!! We matched allpeaks1->allpeaks2 so we now should match allpeaks2->allpeaks1
        deep_windows = get_deepwindows(windows, allpeaks2, allpeaks1, matching_fuzz)
        log.debug(f'Got {len(deep_windows)} deep windows and {len(windows)} windows')

    if not len(windows):
        # patch for empty data
        deep_windows = np.array([[-1, -1]], dtype=(np.int64, np.int64))
    assert np.shape(np.shape(deep_windows))[0] == 2, (
        f'deep_windows shape is wrong {np.shape(deep_windows)}\n{deep_windows}')

    # make array for numba
    unknown_types = np.array(unknown_types)

    # Inner matching
    _match_peaks(allpeaks1, allpeaks2, windows, deep_windows, unknown_types)
    return allpeaks1, allpeaks2


@numba.jit(nopython=True, nogil=True, cache=True)
def _match_peaks(allpeaks1, allpeaks2, windows, deep_windows, unknown_types):
    """See match_peaks_strax where we do the functional matching here"""
    # Loop over left and right bounds for peaks 1 by matching to peaks 2
    for peaks_1_i, (l1, r1) in enumerate(windows):
        peaks_1 = allpeaks1[l1:r1]
        if l1 == r1:
            continue

        for p1_i, p1 in enumerate(peaks_1):
            # Matching the other way around using deep_windows
            l2, r2 = deep_windows[peaks_1_i]

            peaks_2 = allpeaks2[l2:r2]
            matching_peaks = peaks_2
            if len(matching_peaks) == 0:
                pass

            elif len(matching_peaks) == 1:
                # A unique match! Hurray!
                p2 = matching_peaks[0]
                p1['matched_to'] = p2['id']
                p2['matched_to'] = p1['id']
                # Do the types match?
                if p1['type'] == p2['type']:
                    p1['outcome'] = 'found'
                    p2['outcome'] = 'found'
                else:
                    if _in(p1['type'], unknown_types):
                        p2['outcome'] = 'unclassified'
                    else:
                        p2['outcome'] = 'misid_as_s' + str(p1['type'])
                    if _in(p2['type'], unknown_types):
                        p1['outcome'] = 'unclassified'
                    else:
                        p1['outcome'] = 'misid_as_s' + str(p2['type'])
                    # If the peaks are unknown in both sets, they will
                    # count as 'found'.
                matching_peaks[0] = p2
            else:
                # More than one peak overlaps p1
                handle_peak_merge(parent=p1,
                                  fragments=matching_peaks,
                                  unknown_types=unknown_types)

            # matching_peaks is a copy, not a view, so we have to copy
            # the results over to peaks_2 manually Sometimes I wish
            # python had references...
            for i_in_matching_peaks, i_in_peaks_2 in enumerate(range(l2, r2)):
                allpeaks2[i_in_peaks_2] = matching_peaks[i_in_matching_peaks]

        # Match in reverse to detect merged peaks >1 peaks in 1 may
        # claim to be matched to a peak in 2, in which case we should
        # correct the outcome...
        for p2_i, p2 in enumerate(peaks_2):
            selection = peaks_1['matched_to'] == p2['id']
            matching_peaks = peaks_1[selection]
            if len(matching_peaks) > 1:
                handle_peak_merge(parent=p2,
                                  fragments=matching_peaks,
                                  unknown_types=unknown_types)

            # matching_peaks is a copy, not a view, so we have to copy
            # the results over to peaks_1 manually Sometimes I wish
            # python had references...
            for i_in_matching_peaks, i_in_peaks_1 in enumerate(
                    np.where(selection)[0]):
                peaks_1[i_in_peaks_1] = matching_peaks[i_in_matching_peaks]


@numba.jit(nopython=True)
def handle_peak_merge(parent, fragments, unknown_types):
    found_types = fragments['type']
    is_ok = found_types == parent['type']
    is_unknown = _in1d(found_types, unknown_types)
    is_misclass = _combine_and_flip(is_ok, is_unknown)
    # We have to loop over the fragments to avoid making a copy
    for i in range(len(fragments)):
        if is_unknown[i] or is_misclass[i]:
            if _in(parent['type'], unknown_types):
                fragments[i]['outcome'] = 'merged_to_unknown'
            else:
                fragments[i]['outcome'] = 'merged_to_s' + str(parent['type'])
        else:
            fragments[i]['outcome'] = 'merged'
        # Link the fragments to the parent
        fragments[i]['matched_to'] = parent['id']
    if np.any(is_misclass):
        parent['outcome'] = 'split_and_misid'
    # All fragments are either ok or unknown. If more than one fragment
    # is given the same class as the parent peak, then call it "split".
    elif len(np.where(is_ok)[0]) > 1:
        parent['outcome'] = 'split'
    elif np.all(is_unknown):
        parent['outcome'] = 'split_and_unclassified'
    # If exactly one fragment out of > 1 fragments is correctly
    # classified, then call the parent chopped
    else:
        parent['outcome'] = 'chopped'
    # We can't link the parent to all fragments. Link to the largest one
    _max_idx = _argmax(fragments['area'])
    parent['matched_to'] = fragments[_max_idx]['id']


# @numba.jit
# def get_deepwindows(windows, peaks_a, peaks_b, matching_fuzz):
#     """Get matching window of the matched peak versus the original peak"""
#     n_windows = len(windows)
#     _deep_windows = np.ones((n_windows, 2), dtype=np.int64) * -1
#     for window_i, w in enumerate(windows):
#         l1, r1 = w
#         if r1 - l1:
#             match = strax.processing.general._touching_windows(
#                 peaks_a['time'], strax.endtime(peaks_a),
#                 peaks_b[l1:r1]['time'], strax.endtime(peaks_b[l1:r1]),
#                 window=matching_fuzz)
#             if len(match):
#                 this_window = match[0]
#                 _deep_windows[window_i] = this_window
#             else:
#                 # No match
#                 pass
#     return _deep_windows


# --- Numba functions where numpy does not suffice ---
@numba.njit
def _in1d(arr1, arr2):
    """
    Copy np.in1d logic for numba
    Five times faster than numpy #ohyeah
    """
    res = np.zeros(len(arr1), dtype=np.bool_)
    for i1 in range(len(arr1)):
        for v2 in arr2:
            if arr1[i1] == v2:
                res[i1] = 1
                break
    return res


@numba.jit(nopython=True)
def _in(val, arr):
    """
    Check if val is in array
    1.5x faster than val in np.array
    """
    for a in arr:
        if val == a:
            return True
    return False


@numba.jit(nopython=True)
def _argmax(arr):
    """
    Get index of max argument (np.argmax)
    Slightly faster than np.argmax
    """
    m = INT_NAN
    i = INT_NAN
    leng = len(arr)
    for j in range(leng):
        if arr[j] > m:
            m = arr[j]
            i = j
    return i


@numba.njit
def _combine_and_flip(arr1, arr2):
    """Combine the flipped arrays"""
    return _bool_flip(arr1).astype(np.bool_) & _bool_flip(arr2.astype(np.bool_))


@numba.njit
def _bool_flip(arr):
    """Use True ^ array"""
    res = np.zeros(len(arr), dtype=np.bool_)
    for i, a in enumerate(arr):
        if not a or a == 0:
            res[i] = 1
    return res


In [112]:
@numba.jit
def sort_by_time(x):
    if 'channel' in x.dtype.fields:
        return strax.sort_by_time(x)
    else:
        return simple_sort_by_time(x)

In [113]:
@numba.njit
def simple_sort_by_time(x):
    """"""
    if len(x) == 0:
        # Nothing to do, and .min() on empty array doesn't work, so:
        return x
    # I couldn't get fast argsort on multiple keys to work in numba
    # So, let's make a single key...
    sort_key = (x['time'] - x['time'].min())
    sort_i = np.argsort(sort_key)
    return x[sort_i]

In [114]:
%%timeit
np.sort(truth)

1.24 s ± 11.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [120]:
%%timeit
simple_sort_by_time(truth)

136 ms ± 1.23 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [168]:
@numba.jit(cache=True)
def get_deepwindows(windows, peaks_a, peaks_b, matching_fuzz, sort = False):
    """Get matching window of the matched peak versus the original peak"""
    n_windows = len(windows)
    _deep_windows = np.ones((n_windows, 2), dtype=np.int64) * -1
    if sort:
        peaks_a = simple_sort_by_time(peaks_a)
        peaks_b = simple_sort_by_time(peaks_b)
    return _get_deepwindows(windows, peaks_a, peaks_b, matching_fuzz, _deep_windows)

@numba.njit(nogil=True, cache=True, parallel=True)
def _get_deepwindows(windows,
                     peaks_a, 
                     peaks_b, 
                     matching_fuzz, 
                     _deep_windows, 
                     _dt_max = 10e6):
    # Calculate the endtimes once
    peak_a_endtimes = strax.endtime(peaks_a)
    peak_b_endtimes = strax.endtime(peaks_b)
    lenb = len(peak_b_endtimes)
    prev_start=0
    until_end = 0
    for window_i, w in enumerate(windows):
        l1, r1 = w
        if r1 - l1:
            for until_end in range(until_end, lenb):
                if peak_a_endtimes[until_end] > peak_b_endtimes[r1] + _dt_max:
                    break
            match = strax.processing.general._touching_windows(
                peaks_a['time'][prev_start:until_end], peak_a_endtimes[prev_start:until_end],
                peaks_b[l1:r1]['time'], peak_b_endtimes[l1:r1],
                window=matching_fuzz)
            if len(match):
                this_window = match[0] + prev_start
                _deep_windows[window_i] = this_window
                prev_start = max(match[0][0], prev_start)
            else:
                # No match
                pass
    return _deep_windows

In [169]:
_max = 40_000
small_truth, small_peaks = truth[:_max].copy(), peaks[:_max].copy()

In [170]:
windows, allpeaks2, allpeaks1, matching_fuzz = match_peaks(small_truth, small_peaks)

In [ ]:
%%time
_ = get_deepwindows(windows, allpeaks2, allpeaks1, matching_fuzz)

In [165]:
%%timeit
_ = get_deepwindows(windows, allpeaks2, allpeaks1, matching_fuzz)

15.3 s ± 119 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [166]:
%%time
_ = pema.matching.get_deepwindows(windows, allpeaks2, allpeaks1, matching_fuzz)

CPU times: user 14.7 s, sys: 44.9 ms, total: 14.7 s
Wall time: 15.4 s


In [167]:
%%timeit
_ = pema.match_peaks(small_truth, small_peaks)

18.5 s ± 3.98 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [134]:
# %%time
# _ = match_peaks(small_truth, small_peaks)

In [147]:
up = get_deepwindows(windows, allpeaks2, allpeaks1, matching_fuzz)
cur = pema.matching.get_deepwindows(windows, allpeaks2, allpeaks1, matching_fuzz)

43
43
44
45
46
47
48
49
49
51
51
52
52
53
54
54
55
56
59
62
62
62
62
64
65
65
69
69
69
81
82
83
86
86
88
88
91
92
92
94
94
95
98
99
99
101
101
103
103
104
104
107
108
108
109
110
111
111
112
113
113
114
115
115
116
119
119
121
122
124
124
124
124
124
124
124
124
124
124
124
124
125
128
130
130
130
131
131
132
132
132
137
137
139
141
141
141
141
144
145
146
147
149
149
150
151
151
156
158
158
162
162
164
164
167
169
169
169
171
171
171
174
174
174
176
176
176
176
177
177
180
181
182
182
183
183
186
196
198
198
198
199
199
199
200
201
203
203
203
205
206
207
207
208
208
208
210
210
212
212
212
212
214
215
215
216
216
217
219
220
223
224
224
225
226
227
229
229
229
230
231
231
234
234
234
234
236
236
236
236
236
236
236
236
236
236
237
237
240
243
243
244
245
246
247
250
250
252
255
256
257
258
259
259
259
261
262
263
263
264
264
264
264
265
270
272
272
274
274
276
278
278
278
279
281
281
283
283
285
285
287
287
287
291
291
292
294
295
295
295
298
298
299
299
299
302
303
303
306
308
309
3

5446
5448
5448
5449
5450
5451
5451
5453
5454
5454
5454
5454
5454
5454
5454
5457
5457
5458
5459
5460
5460
5460
5462
5462
5463
5464
5464
5465
5465
5469
5471
5471
5472
5472
5473
5474
5476
5477
5479
5481
5481
5481
5482
5482
5485
5487
5487
5487
5489
5490
5490
5492
5492
5492
5496
5496
5496
5499
5500
5500
5502
5505
5505
5507
5507
5509
5509
5511
5511
5511
5512
5513
5515
5515
5517
5517
5517
5518
5520
5520
5521
5521
5524
5527
5527
5527
5528
5529
5529
5529
5534
5535
5536
5536
5538
5539
5540
5542
5542
5542
5544
5544
5546
5547
5547
5548
5550
5550
5551
5551
5552
5553
5554
5554
5556
5557
5558
5559
5561
5561
5561
5563
5563
5565
5565
5566
5566
5567
5570
5570
5571
5571
5573
5573
5574
5574
5574
5575
5576
5577
5579
5581
5582
5583
5584
5585
5586
5590
5590
5591
5591
5593
5594
5595
5597
5597
5599
5599
5599
5600
5601
5602
5603
5604
5605
5606
5606
5606
5607
5610
5610
5611
5612
5612
5614
5616
5616
5616
5616
5616
5617
5619
5619
5620
5622
5624
5624
5624
5624
5624
5625
5625
5625
5627
5629
5629
5630
5631
5632
5632


10434
10435
10436
10437
10438
10439
10441
10442
10443
10444
10444
10445
10447
10447
10447
10447
10452
10452
10456
10457
10457
10457
10461
10461
10462
10463
10466
10466
10468
10470
10470
10470
10471
10474
10474
10477
10479
10479
10480
10482
10484
10484
10484
10485
10487
10489
10489
10491
10491
10492
10494
10495
10495
10495
10496
10497
10500
10500
10502
10502
10503
10506
10509
10509
10509
10510
10510
10511
10512
10512
10513
10513
10515
10515
10515
10515
10517
10518
10518
10520
10520
10522
10522
10522
10523
10523
10524
10526
10526
10527
10528
10530
10530
10531
10531
10531
10532
10534
10536
10536
10536
10536
10537
10539
10541
10541
10541
10541
10542
10542
10542
10550
10550
10550
10555
10556
10556
10558
10559
10561
10562
10563
10564
10564
10568
10570
10570
10570
10572
10572
10573
10573
10573
10579
10581
10581
10584
10584
10584
10585
10585
10585
10587
10590
10590
10590
10592
10592
10592
10594
10594
10594
10595
10596
10598
10598
10598
10599
10599
10602
10604
10604
10604
10608
10608
10609
1061

13561
13561
13563
13564
13564
13564
13565
13566
13567
13568
13570
13571
13572
13578
13582
13582
13584
13585
13587
13587
13588
13588
13592
13592
13593
13595
13597
13597
13599
13599
13599
13599
13599
13599
13600
13602
13603
13605
13606
13607
13608
13608
13608
13609
13609
13612
13612
13614
13614
13617
13619
13619
13620
13621
13622
13623
13623
13625
13625
13625
13625
13625
13625
13625
13626
13626
13626
13626
13628
13628
13631
13631
13631
13632
13632
13634
13635
13635
13637
13637
13637
13638
13648
13648
13649
13650
13650
13650
13659
13659
13659
13661
13661
13663
13663
13664
13665
13667
13668
13670
13671
13671
13672
13672
13673
13675
13676
13679
13679
13680
13680
13683
13684
13684
13684
13686
13686
13688
13688
13688
13692
13693
13695
13696
13696
13696
13696
13696
13696
13696
13696
13696
13696
13696
13696
13700
13701
13701
13701
13701
13701
13701
13701
13702
13702
13703
13705
13706
13706
13708
13709
13709
13709
13711
13711
13711
13712
13712
13715
13718
13718
13719
13721
13721
13721
13725
1372

16991
16993
16993
16994
16994
16995
16996
16997
17002
17003
17003
17005
17006
17008
17009
17010
17012
17012
17013
17014
17014
17016
17016
17016
17019
17020
17022
17025
17025
17029
17030
17039
17041
17041
17041
17041
17041
17041
17041
17041
17041
17041
17041
17045
17045
17045
17046
17046
17046
17046
17047
17047
17049
17050
17051
17053
17053
17053
17053
17053
17053
17055
17056
17056
17057
17057
17059
17060
17061
17062
17064
17066
17066
17068
17068
17068
17069
17070
17072
17072
17072
17074
17074
17074
17076
17076
17078
17078
17081
17081
17081
17081
17081
17081
17081
17081
17081
17081
17082
17083
17084
17084
17084
17084
17089
17089
17091
17091
17094
17094
17096
17096
17097
17097
17100
17101
17101
17101
17103
17103
17104
17104
17105
17106
17107
17108
17108
17110
17110
17111
17113
17114
17114
17115
17117
17118
17124
17126
17126
17126
17127
17128
17130
17130
17130
17130
17132
17132
17138
17142
17143
17143
17145
17146
17146
17147
17148
17148
17148
17155
17156
17157
17159
17159
17162
17162
1716

...

29385
29388
29389
29389
29390
29390
29391
29391
29396
29397
29398
29400
29400
29402
29402
29402
29403
29403
29403
29404
29404
29406
29407
29408
29408
29409
29409
29413
29413
29413
29414
29417
29417
29421
29422
29424
29426
29428
29428
29428
29429
29430
29431
29431
29432
29433
29435
29437
29437
29437
29439
29439
29439
29440
29441
29442
29442
29443
29444
29446
29449
29449
29449
29450
29450
29450
29456
29459
29459
29459
29461
29461
29461
29461
29462
29462
29462
29463
29464
29464
29466
29469
29469
29472
29472
29475
29475
29476
29477
29478
29478
29478
29480
29480
29481
29481
29482
29483
29484
29486
29486
29487
29488
29489
29490
29490
29491
29493
29495
29495
29496
29497
29497
29498
29500
29500
29501
29501
29509
29509
29510
29510
29510
29511
29511
29511
29512
29512
29514
29515
29517
29517
29519
29519
29521
29522
29522
29524
29525
29526
29528
29529
29532
29534
29534
29537
29538
29540
29540
29540
29541
29542
29543
29545
29546
29547
29547
29549
29549
29549
29549
29549
29550
29550
29550
29550
2955

31683
31684
31684
31684
31685
31685
31688
31688
31688
31690
31690
31693
31696
31696
31697
31699
31700
31702
31704
31704
31704
31704
31704
31704
31705
31705
31706
31711
31713
31713
31714
31714
31715
31716
31716
31717
31719
31720
31720
31721
31722
31723
31723
31726
31728
31728
31728
31728
31733
31734
31734
31734
31736
31736
31736
31737
31738
31741
31741
31741
31743
31743
31743
31744
31744
31745
31746
31747
31749
31752
31752
31752
31752
31752
31752
31754
31754
31755
31755
31759
31760
31760
31760
31763
31763
31764
31766
31766
31768
31768
31770
31770
31772
31772
31772
31773
31773
31778
31778
31779
31780
31780
31781
31781
31783
31784
31786
31788
31788
31790
31790
31792
31792
31793
31794
31794
31795
31797
31799
31799
31799
31801
31801
31803
31803
31803
31803
31804
31806
31806
31807
31807
31809
31809
31810
31811
31812
31812
31814
31817
31817
31817
31819
31821
31822
31824
31824
31825
31825
31825
31825
31828
31829
31830
31831
31832
31835
31835
31837
31837
31837
31838
31843
31844
31845
31847
3184

33888
33888
33890
33892
33892
33894
33894
33895
33895
33896
33898
33898
33899
33901
33901
33902
33904
33905
33906
33907
33908
33909
33910
33912
33912
33912
33912
33912
33916
33918
33918
33919
33919
33919
33920
33920
33926
33926
33928
33928
33928
33928
33931
33931
33931
33932
33933
33934
33936
33936
33937
33945
33946
33947
33948
33950
33952
33952
33953
33954
33957
33957
33957
33958
33958
33958
33960
33960
33961
33961
33963
33963
33964
33964
33966
33966
33967
33969
33969
33970
33972
33972
33974
33974
33974
33976
33976
33976
33978
33978
33978
33978
33978
33980
33980
33983
33983
33983
33984
33984
33984
33984
33984
33984
33985
33985
33985
33988
33988
33990
33990
33990
33991
33992
33995
33995
33996
33996
33996
34000
34002
34002
34005
34006
34006
34010
34012
34012
34014
34014
34014
34017
34018
34018
34020
34022
34023
34024
34026
34027
34028
34029
34031
34031
34031
34033
34033
34035
34035
34035
34036
34037
34038
34039
34042
34042
34042
34045
34047
34047
34047
34047
34049
34049
34052
34052
3405

35962
35962
35964
35965
35967
35969
35969
35970
35971
35972
35972
35972
35972
35974
35975
35976
35976
35976
35976
35976
35976
35976
35977
35978
35980
35980
35980
35981
35981
35982
35983
35985
35985
35985
35985
35987
35989
35989
35992
35992
35992
35995
35996
35996
35996
36000
36000
36000
36001
36003
36006
36006
36006
36008
36008
36008
36011
36011
36011
36012
36013
36015
36015
36017
36018
36019
36022
36022
36023
36025
36028
36028
36030
36031
36033
36034
36035
36036
36039
36040
36041
36044
36044
36044
36046
36046
36046
36048
36050
36050
36050
36050
36052
36053
36053
36055
36055
36055
36056
36057
36059
36059
36059
36061
36062
36062
36063
36063
36064
36065
36066
36067
36067
36068
36070
36070
36070
36071
36071
36071
36073
36073
36075
36075
36075
36076
36078
36080
36080
36080
36080
36080
36081
36082
36084
36084
36085
36086
36087
36087
36088
36091
36092
36092
36094
36095
36096
36097
36097
36098
36100
36102
36102
36103
36104
36107
36109
36109
36110
36111
36111
36111
36115
36116
36117
36118
3612

37946
37946
37950
37951
37951
37951
37951
37956
37957
37959
37960
37960
37960
37960
37960
37960
37960
37960
37960
37961
37964
37966
37967
37967
37970
37970
37971
37972
37973
37973
37973
37974
37974
37975
37975
37975
37976
37977
37980
37980
37980
37982
37985
37985
37985
37986
37994
37996
37996
37996
37996
37996
37996
37996
37997
37998
37999
38000
38001
38001
38002
38004
38005
38005
38005
38007
38009
38010
38010
38010
38011
38011
38011
38012
38012
38013
38020
38020
38023
38025
38029
38030
38031
38032
38032
38033
38036
38036
38039
38039
38039
38041
38041
38041
38041
38041
38041
38041
38041
38041
38044
38044
38046
38046
38048
38051
38051
38054
38055
38055
38056
38057
38058
38059
38059
38060
38062
38063
38065
38066
38066
38066
38066
38066
38066
38066
38069
38069
38069
38070
38071
38073
38073
38073
38073
38074
38075
38076
38077
38078
38078
38078
38079
38079
38079
38081
38081
38083
38083
38083
38084
38086
38087
38087
38088
38088
38090
38092
38092
38094
38094


In [148]:
np.all(up == cur)

True

In [137]:
up

array([[ 0,  0],
       [ 0,  0],
       [ 0,  0],
       ...,
       [-1, -1],
       [-1, -1],
       [-1, -1]])

In [138]:
cur

array([[ 0,  1],
       [ 1,  2],
       [ 2,  3],
       ...,
       [-1, -1],
       [-1, -1],
       [-1, -1]])